# Introduction to Testing

In [1]:
from fastcore.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
from utils import view_source_code
from datetime import datetime, timedelta, date
import time

In [2]:
path = Path('../data')

## Background

In Chapter 1 we created models and created actions we want to take for multiple approaches.  The question now is, how do we know if they are profitable?  How should we measure them?  How do we know if we simply got lucky, or if they are reliable?

As we mentioned in chapter 2, testing is the most important part of the process.  If done well you have a good way to determine what strategies should be implemented, and if done poorly you run the risk of implementing non-profitable strategies.  I believe you should strive to never sacrifice testing principles because effective testing is your **only** objective indication to whether you are doing something useful or not.  Without effective testing you are "flying blind".

This chapter will lay the groundwork and cover the basics of testing.  The goal of this chapter is to introduce concept and the three core things that need to be carefully considered for effective testing.

1. What data should you use for testing?
1. What metric should you use for testing?
1. What test should you use?

In this chapter we are going to walk through the concepts to see each step and understand the importance.  In the next chapter we are going apply our knowledge to build a better structured solution and framework that we can use through the remainder of the book.

## What data should you use for testing?

The first question we have to ask is what data to we use for testing?  Ideally we have 3 subsets of our data (training, validation, and test).  Let's go through what they are used for and why they are important.

### Training Set

The training set is unique because it has no restrictions on what we can do with it.  We can look at any piece of data in it.  We can normalize data using values in the training set.  We can train machine learning models on the training set.  This is often the largest subset of our data.

This training set is pretty explanatory - we use this for understanding our data and developing our model.  

We can load it in using the same method as we did in chapter 1.

In [3]:
raw = pd.read_csv(path/'eod-quotemedia.csv',parse_dates=['date'])
df = raw.pivot(index='date', columns='ticker',values='adj_close')
train = df.loc[:pd.Timestamp('2017-1-1')]

### Validation Set

The goal of creating a trading strategy is to have it perform well on data that it was not developed using.  We may use data from 2015 - 2020 to create a trading strategy, but the goal is to apply it to 2021 and 2022 to make a profit.

Because we want our model to perform on *unseen* data, we create some restriction to how we use the validation set.  We do not train any models on it, and we do not use statistics or data from the validation set when creating our model.  It's data our model has never seen.  The validation set is something we can only use to see how well our strategy or model performs.  

The entire purpose of the validation set is to give us unseen data to evaluate our approaches on.  By having this separate validation set we can more accurately determine what works and what doesn't.

We can get our validation set using the same method as we did in chapter 1.

In [4]:
valid = df.loc[pd.Timestamp('2017-1-1'):]

### Test Set


The Test set is very similar to the validation set, but it takes things a step further.  It has further restrictions in that is is the final model step before deployment.  The main difference is how often you can use it.  For the validation set, you can test anything on the validation set as many times as you want.  For the test set you only get to look at the test set once for your particular approach.

For example, you may try 300 different approaches and parameter changes to your strategy to see what works best.  You can check the profitability on each of them using the validation set.  Then once you have chosen a strategy, you do a final check to ensure it also performs on the test set.  Once you have done that you need a new test set or your project is over.

The reason this is important is that you want to ensure that you didn't get lucky and find a configuration out of your 300 attempts that just happens to work on the validation set but doesn't work elsewhere.  If you try enough combinations eventually you will find something that works, but the test set gives you confidence that your model works because it's a good strategy and not that you just tried enough things to find something that works on coincidence.


:::{note} Many people re-use or have more lax rules on the test set.  Many people do not use one at all.  In this text I am laying out the ideal state I believe we should strive for.  If you choose to loosen these restrictions on the test set or do without one, I would strongly encourage you to think hard about it.


To get our test set, we could have split our initial data into 3.  Because we are a bit concerned about survivorship bias, let's pull a new test set that uses recent data to and test how these strategies would perform over the last year and a half.

We need to get adjusted close price.  There are a variety of services that have APIs to pull from, I have picked polgygon to use here because it's free for what we need.

:::{note} We are using a free api key and putting the key in this notebook in an effort to show everthing to the reader, but best practice would be to read the api key from an environment variable.  From the polygon docs it will by default pull the api key from the `POLYGON_API_KEY` environment variable, then you would initiate the client with so that no credentials are exposed.
`client = RESTClient()`

In [5]:
polygon_free_api_key = 'wUv2tpS05klv9ebAQKyLD610FBWllpan'
client = RESTClient(polygon_free_api_key)

In [6]:
if not (path/'polytest_eod-quotemedia.csv').exists():
    dfs = L()
    errors = L()
    for ticker in valid:
        try:
            aggs = client.get_aggs(ticker, 1, "day", "2021-01-01", "2022-05-31",adjusted=True)
            close = {ticker:[o.close for o in aggs]}
            
            # Convert millisecond time stamp to date
            date = L(o.timestamp/1e3 for o in aggs).map(datetime.fromtimestamp)
            dfs.append(pd.DataFrame(close,index=date))
        except:
            errors.append(aggs)
            print(f"FAILURE: {ticker}")
        
        # Free api gives 5 API calls / minute - so we need to pace our api calls!
        time.sleep(60/5)
    df_test = pd.concat(dfs,axis=1)
    df_test.to_csv(path/'polytest_eod-quotemedia.csv')

df_test = pd.read_csv(path/'polytest_eod-quotemedia.csv',index_col=0,parse_dates=True)

In [7]:
df_test.iloc[:5,:5]

,A,AAL,AAP,AAPL,ABBV
2021-01-04,118.64,15.13,157.34,129.41,105.41
2021-01-05,119.61,15.43,157.17,131.01,106.50
2021-01-06,122.89,15.52,166.25,126.60,105.58
2021-01-07,126.16,15.38,167.67,130.92,106.71
2021-01-08,127.06,15.13,170.06,132.05,107.27


## What metric should you use for testing?

Now that we understand what data we will use for testing, let's start figuring out how well our first model from chapter 1 performs.

The next step is to figure out an appropriate metric.  There are a variety of ways to measure this and we will walk through a few first steps in this section

### Dollars

Let's take our first model from chapter 1 and measure how well it does in terms of dollars.  After all dollars is what we want to make, so it seems like a reasonable starting point.

In [8]:
from SimpleTimeSeries import get_momentum_actions

In [9]:
transactions = pd.DataFrame(columns=['open_date','ticker','action','close_date'])

valid_mom = get_momentum_actions(valid,28,0.08)
for dte,vals in valid_mom.iloc[:,:5].iterrows():    
    for val in vals[vals.values != ''].items():
        row = {'open_date':dte.date(),'ticker':val[0],'action':val[1]}
        transactions = pd.concat([transactions,pd.DataFrame(row,index=[0])])


No we have a dataframe with all the positions we are going to take and when to take them.  But we are missing one crucial piece!  When should we close those positions.  We cannot make money by simplying buying a stock (ignoring dividends for now) - the profit comes when we actually close the position and sell the stock.  Let's close the position 28 days after opening.

In [10]:
transactions['close_date'] = transactions.open_date + timedelta(28)
transactions = transactions.loc[transactions.open_date < (transactions.open_date.max() - timedelta(28))]


Next we need to get the stock price on the date of our initial action when we open to position, as well as when we close our position.  Let's start with the price on the day we open.

In [11]:
df_valid_long = valid.melt(var_name='ticker',value_name='adj_close',ignore_index=False).reset_index()
df_valid_long.columns = ['dte','ticker','adj_close']

In [12]:
transactions['open_date'] = pd.to_datetime(transactions.open_date)
df_valid_long['dte'] = pd.to_datetime(df_valid_long.dte)
pd.merge(left=transactions,left_on=['open_date','ticker'],
         right=df_valid_long,right_on=['dte','ticker'],
         how='left').head(10)

,open_date,ticker,action,close_date,dte,adj_close
0,2017-02-14,A,Buy,2017-03-14,2017-02-14,49.703267
1,2017-02-14,AAPL,Buy,2017-03-14,2017-02-14,132.397351
2,2017-02-15,AAPL,Buy,2017-03-15,2017-02-15,132.877833
3,2017-02-16,A,Buy,2017-03-16,2017-02-16,50.147135
4,2017-02-16,AAPL,Buy,2017-03-16,2017-02-16,132.716038
5,2017-02-17,AAPL,Buy,2017-03-17,2017-02-17,133.083754
6,2017-02-18,AAPL,Buy,2017-03-18,NaT,NaN
7,2017-02-22,AAPL,Buy,2017-03-22,2017-02-22,134.446754
8,2017-02-23,AAPL,Buy,2017-03-23,2017-02-23,133.878020
9,2017-02-24,AAP,Short,2017-03-24,2017-02-24,156.842676


Uh oh - We have a join that isn't working correctly and get `NaT` and `NaN`!  We created our model assuming that we could make transactions any day we want, but the stock market is not open every day.  There are limitations to when we can trade openly in the stock market we need to start accounting for.  

When we trade using the adjusted close price we added a day because we wouldn't be able to actually place the trade until the following day.  If that day ended up being a Saturday in reality we would have to wait until Monday to place that trade (assuming that monday isn't a holiday).  

Let's fix that by getting the next available trading day for each date.  Because we know this same thing applies to our `close_date`, we will fix it there as well.

In [13]:
unique_dates = L(o.date() for o in valid.index)
unique_dates.sort()

In [14]:
def get_next_trading_day(dte,unique_dates):
    return unique_dates.filter(lambda x: pd.Timestamp(x) >= pd.Timestamp(dte))[0]

In [15]:
f = bind(get_next_trading_day,unique_dates=unique_dates)
transactions['open_date'] = transactions.open_date.apply(f)
transactions['close_date'] = transactions.close_date.apply(f)

In [16]:
transactions['open_date'] = pd.to_datetime(transactions.open_date)
transactions['close_date'] = pd.to_datetime(transactions.close_date)


Now we can merge in the price correctly!

In [17]:
transactions = pd.merge(left=transactions,left_on=['open_date','ticker'],
                         right=df_valid_long,right_on=['dte','ticker'],
                          how='left')
transactions = pd.merge(left=transactions,left_on=['close_date','ticker'],
                         right=df_valid_long,right_on=['dte','ticker'],
                          how='left',
                          suffixes=('_atOpen','_atClose'))

In [18]:
transactions.head(10)

,open_date,ticker,action,close_date,dte_atOpen,adj_close_atOpen,dte_atClose,adj_close_atClose
0,2017-02-14,A,Buy,2017-03-14,2017-02-14,49.703267,2017-03-14,51.498464
1,2017-02-14,AAPL,Buy,2017-03-14,2017-02-14,132.397351,2017-03-14,136.290237
2,2017-02-15,AAPL,Buy,2017-03-15,2017-02-15,132.877833,2017-03-15,137.731683
3,2017-02-16,A,Buy,2017-03-16,2017-02-16,50.147135,2017-03-16,52.327016
4,2017-02-16,AAPL,Buy,2017-03-16,2017-02-16,132.716038,2017-03-16,137.957216
5,2017-02-17,AAPL,Buy,2017-03-17,2017-02-17,133.083754,2017-03-17,137.270813
6,2017-02-21,AAPL,Buy,2017-03-20,2017-02-21,134.044718,2017-03-20,138.712259
7,2017-02-22,AAPL,Buy,2017-03-22,2017-02-22,134.446754,2017-03-22,138.673036
8,2017-02-23,AAPL,Buy,2017-03-23,2017-02-23,133.878020,2017-03-23,138.182748
9,2017-02-24,AAP,Short,2017-03-24,2017-02-24,156.842676,2017-03-24,149.325422


In [19]:
buy_transactions = transactions.loc[transactions.action == 'Buy']
short_transactions = transactions.loc[transactions.action == 'Short']

buy_profit = (buy_transactions.adj_close_atClose - buy_transactions.adj_close_atOpen).sum()
short_profit= -(short_transactions.adj_close_atClose - short_transactions.adj_close_atOpen).sum()

buy_profit,short_profit,buy_profit+short_profit

(56.40339746321577, 33.86091030275433, 90.2643077659701)

Great!  So according to our validation set we made $90 profit (pre-tax).  We could buy/short in higher volumes (ie Buy = 10x buys, Short = 10x shorts) to make this profit larger.

However this really isn't enough information to determine whether that is a good idea of feasible.  I would love to loan someone \\$100 if they would give me \\$190 dollars back.  I would hate to loan someone \\$10,000 on the promise that they would pay me \\$10,090 back.  The reward just wouldn't be worth the risk.

Let's see if we can come up with a better metric.

### Percent Return

Instead of measuring raw dollars, lets consider how much money (capital) we needed in order to make that $90 profit.  To do this we need to keep track of our money more carefully than just looking at how much we made at the end.  Let's track how much we have invested after each transaction.

In [20]:
transactions.head()

,open_date,ticker,action,close_date,dte_atOpen,adj_close_atOpen,dte_atClose,adj_close_atClose
0,2017-02-14,A,Buy,2017-03-14,2017-02-14,49.703267,2017-03-14,51.498464
1,2017-02-14,AAPL,Buy,2017-03-14,2017-02-14,132.397351,2017-03-14,136.290237
2,2017-02-15,AAPL,Buy,2017-03-15,2017-02-15,132.877833,2017-03-15,137.731683
3,2017-02-16,A,Buy,2017-03-16,2017-02-16,50.147135,2017-03-16,52.327016
4,2017-02-16,AAPL,Buy,2017-03-16,2017-02-16,132.716038,2017-03-16,137.957216


Each row of our transaction table is actually 2 actions.
1. Action we take to open a position
1. Action we take to close the position

Let's change this to be 1 action per row so we can more easily look at things sequentially.

In [21]:
open_trans = transactions[['open_date','ticker','action','adj_close_atOpen']].copy()
open_trans['type'] = "Open"

close_trans = transactions[['close_date','ticker','action','adj_close_atClose']].copy()
close_trans['action'] = close_trans['action'].apply(lambda x: "Sell" if x=="Buy" else "Return")
close_trans['type'] = "Close"


In [22]:
transactions = pd.DataFrame(np.concatenate([open_trans.values,close_trans.values]),
                        columns=['trans_date','ticker','action','price','type'])
transactions.sort_values('trans_date',inplace=True)
transactions.head()

,trans_date,ticker,action,price,type
0,2017-02-14,A,Buy,49.703267,Open
1,2017-02-14,AAPL,Buy,132.397351,Open
2,2017-02-15,AAPL,Buy,132.877833,Open
3,2017-02-16,A,Buy,50.147135,Open
4,2017-02-16,AAPL,Buy,132.716038,Open


Great!  So now we can go row by row by execute each action sequentially.  If `type==Open` we are opening a position (spending money), if `type==Close` we are closing the position (making or losing money).

In [23]:
transactions.action.unique()

array(['Buy', 'Short', 'Sell', 'Return'], dtype=object)

+ I spend $100 to buy a stock.
+ I sell that stock for $110

I have invested 100, and have revenue of 110, for 10 dollar profit


+ I spend 100 to sell a stock
+ I reuturn that stock for 110

I invested 110, and have revenue of 100 for a net loss of 10

+ I spend 100 to sell a stock
+ I reuturn that stock for 90

I invested 90, and have revenue of 100 for a net profit of 10

In [24]:
invested_buy = L(0)
revenue_buy = L(0)
invested_short = L(0)
revenue_short = L(0)

for idx,vals in transactions.iterrows():
    if (vals.type=="Open") and (vals.action=="Buy"):
        invested_buy.append(invested_buy[-1]+vals.price)
        revenue_buy.append(revenue_buy[-1])
        invested_short.append(invested_short[-1])
        revenue_short.append(revenue_short[-1])

    if (vals.type=="Close") and (vals.action=="Sell"):
        revenue_buy.append(revenue_buy[-1]+vals.price)
        invested_buy.append(invested_buy[-1])
        invested_short.append(invested_short[-1])
        revenue_short.append(revenue_short[-1])

    if (vals.type=="Open") and (vals.action=="Short"):
        revenue_short.append(revenue_short[-1]+vals.price)
        invested_buy.append(invested_buy[-1])
        revenue_buy.append(revenue_buy[-1])
        invested_short.append(invested_short[-1])

    if (vals.type=="Close") and (vals.action=="Return"):
        invested_short.append(invested_short[-1]+vals.price)
        invested_buy.append(invested_buy[-1])
        revenue_buy.append(revenue_buy[-1])
        revenue_short.append(revenue_short[-1])

In [25]:
transactions['invested_buy'] = invested_buy[1:]
transactions['revenue_buy'] = revenue_buy[1:]
transactions['invested_short'] = invested_short[1:]
transactions['revenue_short'] = revenue_short[1:]

transactions['profit_buy'] = transactions.revenue_buy - transactions.invested_buy
transactions['profit_short'] = transactions.revenue_short - transactions.invested_short

transactions['ttl_invested'] = transactions.invested_buy + transactions.invested_short
transactions['ttl_revenue'] = transactions.revenue_buy + transactions.revenue_short
transactions['ttl_profit'] = transactions.profit_buy + transactions.profit_short

In [26]:
transactions.iloc[-1,-3:]

ttl_invested    10808.976839
ttl_revenue     10899.241147
ttl_profit         90.264308
Name: 221, dtype: object

So now we can see several interesting things.  First, we can validate that we get the same amount of profit for buy, short, and overall (~\\$90) so we know our math did the same thing.

Next we can see that in order to make that \\$90 of profit we had to invest \\$10,808 dollars!  Let's calculate what our `percent return` is.

In [27]:
transactions.ttl_revenue.iloc[-1]

10899.241146966167

In [28]:
end_values = transactions.iloc[-1,:]
(end_values.ttl_revenue - end_values.ttl_invested) / end_values.ttl_invested

0.00835086512893754

Ok, so we see we got 0.8\% return on our investment.  That's definitely better than 0, but how do we know if that's good enough to use?

A good first step is for us to compare to an index.  Let's see how the S&P 500 did in that time to determine if we beat the market.

In [29]:
print(f"Time Range: {transactions.trans_date.min().date()} - {transactions.trans_date.max().date()}")

Time Range: 2017-02-14 - 2017-06-30


+ On 2/14 the S&P 500 had an ajusted close price of 2,337.58
+ On 6/30 the S&P 500 had an ajdusted close price of 2,423.41

If we use that to caculate percent return we get:

$$\frac{(2423.41 - 2337.58)}{2337.58} = 3.7\%$$

And compare that to out previous return 

$$\frac{3.7}{0.8} = 4.625$$

So if we had just invested in the S&P 500 we would have done over 4.5x better!  We may have made a profit, but we could have saved time and taken on less risk while making more money by simply investing the S&P 500.  Our investment strategy isn't looking so good after all!

### Log Return

## Statistical Tests